# Usage (quick version)

## Preparation

In [ ]:
# Standard users
# !pip install covsirphy
# Developers (Note: this notebook is in example directory)
import os
os.chdir("../")

In [ ]:
import covsirphy as cs
cs.__version__

## Dataset preparation
Download the datasets to "input" direcotry and load them.

If "input" directory has the datasets, `DataLoader` will load the local files. If the datasets were updated in remote servers, `DataLoader` will update the local files automatically.
Download the datasets to "input" direcotry and load them.

If "input" directory has the datasets, `DataLoader` will load the local files. If the datasets were updated in remote servers, `DataLoader` will update the local files automatically.

In [ ]:
data_loader = cs.DataLoader("input")
# The number of cases (JHU style)
jhu_data = data_loader.jhu(verbose=True)
# Population in each country
population_data = data_loader.population(verbose=True)
# Government Response Tracker (OxCGRT)
oxcgrt_data = data_loader.oxcgrt(verbose=True)

### The number of cases (JHU style)

In [ ]:
# Description/citation
print(jhu_data.citation)

In [ ]:
print(data_loader.covid19dh_citation)

In [ ]:
# Raw data
jhu_data.raw.tail()

In [ ]:
# Cleaned data
jhu_data.cleaned().tail()

In [ ]:
# Subset for a country
jhu_data.subset("Japan", province="Tokyo").tail()

In [ ]:
# Total
total_df = jhu_data.total()
total_df.tail()

In [ ]:
# Plot the total values
cs.line_plot(total_df[["Infected", "Fatal", "Recovered"]], "Total number of cases over time")

### (Optional) Replace the dataset in Japan


In [ ]:
jhu_data.subset("Japan").tail()

In [ ]:
japan_data = data_loader.japan()
print(japan_data.citation)

In [ ]:
jhu_data.replace(japan_data)
jhu_data.subset("Japan").tail()

### Population in each country

In [ ]:
# Description/citation
print(population_data.citation)

In [ ]:
# Raw data (the same as jhu_data)
population_data.raw.tail()

In [ ]:
# Cleaned data
population_data.cleaned().tail()

In [ ]:
population_data.value("Japan", province=None)

In [ ]:
# Register population value of Tokyo in Japan
# https://www.metro.tokyo.lg.jp/tosei/hodohappyo/press/2020/06/11/07.html
population_data.update(14_002_973, "Japan", province="Tokyo")
# population_data.value("Japan", province="Tokyo")

### Government Response Tracker (OxCGRT)

In [ ]:
# Description/citation
print(oxcgrt_data.citation)

In [ ]:
# Raw data (the same as jhu_data)
oxcgrt_data.raw.tail()

In [ ]:
# Cleaned data
oxcgrt_data.cleaned().tail()

In [ ]:
# Subset for a country
oxcgrt_data.subset(country="Japan").tail()

In [ ]:
# Subset for a country (using ISO3 code)
oxcgrt_data.subset(iso3="JPN").tail()

## Scenario analysis
As an example, we will analysis the number of cases in Japan.

In [ ]:
scenario = cs.Scenario(jhu_data, population_data, country="Japan", province=None)

### Check records

In [ ]:
record_df = scenario.records()
record_df.tail()

### S-R trend analysis

In [ ]:
scenario.trend(set_phases=True)
scenario.summary()

### Hyperparameter estimation of ODE models
As an example, use SIR-F model.
We can use SIR, SIRD, (SIRFV), (SEWIRF).

In [ ]:
scenario.estimate(cs.SIRF)

In [ ]:
scenario.summary()

### Accuracy of estimation

In [ ]:
scenario.estimate_accuracy(phase="1st")

### Get parameter value

In [ ]:
scenario.get("Rt", phase="4th")

### Show parameter history

In [ ]:
df = scenario.param_history(targets=["rho", "sigma"])
df.T

### Show the history of reproduction number

In [ ]:
df = scenario.param_history(targets=["Rt"], divide_by_first=False, show_box_plot=True)
df.T

### Simulate the number of cases

In [ ]:
scenario.clear(name="Main")
scenario.add(name="Main", end_date="01Jan2021")
main_sim_df = scenario.simulate(name="Main")
main_sim_df.tail()

### Simulate another scenario
If $\sigma$ will be double in 30 days, how many cases will be?

In [ ]:
sigma_current = scenario.get("sigma", name="Main", phase="last")
sigma_current

In [ ]:
sigma_new = sigma_current * 2
sigma_new

In [ ]:
scenario.clear(name="Medicine")
scenario.add(name="Medicine", days=30, sigma=sigma_current)
scenario.add(name="Medicine", end_date="01Jan2021", sigma=sigma_new)
scenario.summary()

In [ ]:
scenario.simulate(name="Medicine").tail()

### Compare the scenarios

In [ ]:
scenario.describe()